In [1]:
import duckdb
import os
from tqdm import tqdm
import pandas as pd
import torch
from textsum.summarize import Summarizer

In [5]:
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    print("GPU encontrada:", device)
else:
    print("Nenhuma GPU disponível")

Nenhuma GPU disponível


In [2]:
con = duckdb.connect(database='jus4.duckdb', read_only=False)

In [6]:
summarizer = Summarizer(
    model_name_or_path="pszemraj/long-t5-tglobal-base-16384-book-summary",
    num_beams=4,
    force_cache=True,
    #optimum_onnx=True,
    compile_model=True
)

11/25/2023 07:17:57 INFO Compiling model
11/25/2023 07:18:00 INFO Loaded model pszemraj/long-t5-tglobal-base-16384-book-summary to cpu
11/25/2023 07:18:00 INFO Forcing use_cache to True


In [8]:
data = con.execute("SELECT * FROM decisions_description").df()
data

,ID,content,summarize
0,145522785,EMENT\nADMINISTRATIVE. COMPETITION POLICY CONF...,None
1,145501377,DECISION > It is treated of habeas corpus impe...,None
2,145496750,DECISION<br> It is considered a special appeal...,None
3,146037675,DECISION<br> It is considered an ordinary appe...,None
4,145049719,DECISION<br> It is treated of habeas corpus im...,None
...,...,...,...
267,145633556,DECISION is dealt with by the STATE OF SÃO PAU...,None
268,143697533,DECISION<br> This is the case of the Court of ...,None
269,145050021,DECISION<br> This is the case of a misconduct ...,None
270,145717143,DECISION<br>1. Take care of agravo interposed ...,None


In [12]:
for index, element in tqdm(enumerate(data["content"]), total=data["ID"].size):
  out_str = summarizer.summarize_string(element)
  con.execute("""
      UPDATE decisions_description
      SET summarize = $out_str
      WHERE ID = $id;
      """,{
        'out_str': out_str,
        'id': int(data["ID"][index]),
      })

con.commit()

  0%|          | 0/272 [00:00<?, ?it/s]

Generating Summaries:   0%|          | 0/10 [00:00<?, ?it/s]

/home/jadson/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/modeling_utils.py:861: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
  0%|          | 0/272 [14:25<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
con.close()